In [1]:
using Base.Dates
using AxisArrays
using Vega

In [2]:
function gen_days(nyears)
    alldays = [Date(now()-Year(nyears)):Date(now());]
    daynums = [dayofweek(d) for d in alldays]
    #weekdays = alldays[daynums .< 6]
    weekdays = alldays # there's no easy way to have weekdays as axis of an AxisArray
    #ndays = length(weekdays)

    ## delete random holidays (15 per year)
    #holidays = randperm(ndays)[1:(15 * nyears)]
    #daymask = trues(ndays)
    #daymask[holidays] = false
    #weekdays[daymask]
end

function gen_volatility(dayavgs, volatility)
    ndays = size(dayavgs, 2)
    out = copy(dayavgs)
    for d in 1:ndays
        out[:,d] += out[:,d] .* volatility .* randn(500)
    end
    abs(out)
end

function gen_prices(nstocks, ndays)
    t1prices = repmat([50.:4.3:650.;], 4)[randperm(nstocks)]
    t2prices = repmat([-50.:8.7:800.;], 6)[randperm(nstocks)]
    incrprices = (t2prices .- t1prices) / ndays

    dayavgs = Array(Float64, nstocks, ndays)
    dayavgs[:,1] = t1prices
    for d in 2:ndays
        prevday = dayavgs[:,d-1]
        dayavgs[:,d] = prevday .+ incrprices
    end
    dayavgs
end

function generate_data()
    dateaxis = Axis{:date}(Date(now()-Year(10)):Date(now()))
    scriptaxis = Axis{:script}(1:500)
    priceaxis = Axis{:price}([:open, :high, :low, :close])

    ndays = length(dateaxis)
    nstocks = length(scriptaxis)
    dayavgs = gen_prices(nstocks, ndays)
    volatility = repmat([0.0:0.5:20.0;], 30)[1:nstocks] / 100

    opens = gen_volatility(dayavgs, volatility)
    closes = gen_volatility(dayavgs, volatility)
    highs = gen_volatility(dayavgs, volatility)
    lows = gen_volatility(dayavgs, volatility)
    highs = max(opens, closes, highs, lows)
    lows = min(opens, closes, highs, lows)
    
    A = AxisArray(zeros(length(scriptaxis), length(dateaxis), length(priceaxis)), scriptaxis, dateaxis, priceaxis);
    for script in 1:500
        A[script, :, :open] = opens[script, :]
        A[script, :, :close] = closes[script, :]
        A[script, :, :low] = lows[script, :]
        A[script, :, :high] = highs[script, :]
    end
    A
end

generate_data (generic function with 1 method)

In [3]:
data = generate_data();

In [4]:
function plotprices(data::AxisArray, stock, pricecol)
    prices = data[stock, :, pricecol]
    L = length(prices)
    lineplot(x=1:L, y=reshape(prices, L))
end

function plotprices(data::AxisArray, stock)
    prices = data[stock, :]
    L = length(prices)
    lineplot(x=1:L, y=reshape(prices, L))
end

plotprices (generic function with 2 methods)

In [5]:
plotprices(data, 1, :low)

In [6]:
function shiftaxis{T,N,D,Ax}(data::AxisArray{T,N,D,Ax}, id, by)
    d = axisdim(data, Axis{id}())
    shiftaxis(data, axes(data, d), by)
end

function shiftaxis{T,N,D,Ax,name,S}(data::AxisArray{T,N,D,Ax}, ax::Axis{name,S}, by)
    allaxes = [data.axes...]
    d = axisdim(data, ax)
    allaxes[d] = Axis{name,S}(ax.val + by)    
    AxisArray(data.data, allaxes...)
end

function dailyreturns(data::AxisArray)
    shifted1day = shiftaxis(data, :date, Day(1));
    d1 = shifted1day[:,:,:open]
    d0 = shifted1day[:,:,:close]
    axes = d1.axes
    AxisArray(100 * (d1 .- d0) ./ d1, axes[1], axes[2])
end

@generated function AxisArrays.axisvalues{T,N,D,Ax,Aname}(A::AxisArray{T,N,D,Ax}, ax::Axis{Aname})
    d = axisdim(A, ax)
    At = Ax.parameters[d].parameters[2]
    quote
        (axes(A)[$d].val)::$At
    end
end

function window{T,N,D,Ax}(data::AxisArray{T,N,D,Ax}, width_ndays::Int, step_ndays::Int, fn)
    avs, avd = axisvalues(data)
    
    d1, d2 = extrema(avd)
    d2 = d2 - Day(width_ndays)
    avd = d1:Day(step_ndays):d2
    datesax = Axis{:date}(avd)
    
    res = AxisArray(zeros(length(avs), length(avd)), axes(data)[1], datesax)
    for s in avs
        d = d1
        while d <= d2
            # TODO: too expensive to do windowing this way (too many allocations)
            # must take a function and apply it just on the data
            v = data[s, d..(d+Day(width_ndays))]
            res[s,d] = fn(v.data)
            d += Day(step_ndays)
        end
    end
    res
end

axisvalues (generic function with 4 methods)

In [7]:
drets = dailyreturns(data)

2-dimensional AxisArray{Float64,2,...} with axes:
    :script, 1:500
    :date, 2006-05-12:1 day:2016-05-12
And data, a 500x3654 Array{Float64,2}:
   0.0          0.0         0.0        0.0        …    0.0         0.0      
  -0.946508    -0.341709   -0.206254   0.159926       -0.117225   -0.845781 
   0.192591     0.23042     1.59677    0.198797        0.749756   -0.865755 
   1.51277     -0.481134    0.65397   -0.0177308       0.484196   -0.945699 
  -2.03269     -0.330941   -0.706649  -3.14544        -7.55109    -5.15325  
  -5.88077     -0.731425   -1.47483   -0.883292   …    0.639969   -0.221408 
  -5.69642     -1.01673    -0.707491  -1.58674        -1.67052     4.86043  
   4.65067      7.94474    -0.872369  -2.51597        -0.423189    3.11881  
  -2.5446      -2.43613    -0.264282  -4.46221         5.33685    -8.85249  
   3.15763     -4.26175    -3.37591   -4.44062       -13.6246      3.11527  
  -3.19542     -3.85175     1.02213    5.99349    …   -1.2395     -8.23667  
   2.3

In [8]:
plotprices(drets, 4)

In [9]:
movingavg{T,N,D,Ax}(data::AxisArray{T,N,D,Ax}, width_ndays::Int, step_ndays::Int=1) = 
    window(data, width_ndays, step_ndays, mean)

volatility{T,N,D,Ax}(data::AxisArray{T,N,D,Ax}, width_ndays::Int, step_ndays::Int=1)=
    window(data, width_ndays, step_ndays, std)

volatility (generic function with 2 methods)

In [10]:
movavg = movingavg(drets, 5)

2-dimensional AxisArray{Float64,2,...} with axes:
    :script, 1:500
    :date, 2006-05-12:1 day:2016-05-07
And data, a 500x3649 Array{Float64,2}:
   0.0        0.0          0.0         0.0        …    0.0          0.0      
  -0.143985   0.00654505   0.0482721   0.0190438       0.106845     0.0372858
   0.575045   0.923667     0.607845    0.16483        -0.80876     -0.560952 
   0.361641   0.573258     0.995535    0.858197        1.03071      0.765531 
  -0.16277    0.347141     0.686834    0.849648       -1.93848     -2.66943  
  -1.35164    0.422936     0.673031    1.54357    …    1.25167      0.955426 
  -1.41109   -1.23825     -2.85854    -4.10839         1.99219      2.12924  
   1.853      1.00161     -1.28935    -2.27369        -3.38774     -2.63908  
  -1.05912   -0.776693    -1.29763    -1.84544        -1.8176      -2.98361  
  -5.03767   -6.57208     -5.09709    -5.51213        -2.9423      -2.5631   
   1.44922    4.18129      3.06744     4.27923    …    2.32088     -0.426

In [11]:
plotprices(movavg, 22)

In [12]:
volat = volatility(movavg, size(movavg,2)-1, size(movavg,2)-1)

2-dimensional AxisArray{Float64,2,...} with axes:
    :script, 1:500
    :date, 2006-05-12:3648 days:2006-05-12
And data, a 500x1 Array{Float64,2}:
  0.0     
  0.287864
  0.600107
  0.888684
  1.15128 
  1.45365 
  1.69926 
  1.99065 
  2.36599 
  2.6239  
  2.89455 
  2.96943 
  3.53906 
  ⋮       
 11.6409  
 11.8169  
 12.977   
 15.2242  
  0.0     
  0.286942
  0.562968
  0.857179
  1.14555 
  1.44609 
  1.76419 
  1.98361 

In [13]:
minimum(volat), median(volat), maximum(volat)

(0.0,5.838061834754816,15.224243046614507)

In [25]:
mask = 12 .< volat .< 14
maskedvolat = volat[mask]
x = axisvalues(maskedvolat, axes(maskedvolat)[axisdim(maskedvolat, Axis{:script}())])
y = reshape(copy(maskedvolat.data), length(maskedvolat)) .- 12
barplot(x=x, y=y)